In [1]:
import numpy as np
import pandas as pd

### How do we predict what rating a user is going to give?
1) Business side:
    - location
    - type
    - hours
    - parking availability
    - ambience
    - takeout?
2) User side:
    - how long on yelp
    - elite status
    - review count
    - useful/funny/cool
    - home

# TODO
    - Matthew
        - isolate restaurants in business_df
        - preprocessing: marking reviews as good or bad
    - feature extraction
        - hometown
        - number of tips by user

# Next
    - what happens if we remove "inactive" users

        

In [5]:
business_df = pd.read_csv("sample_business.csv")
checkin_df = pd.read_csv("sample_checkin.csv")
review_df = pd.read_csv("sample_review.csv")
tip_df = pd.read_csv("sample_tip.csv")
user_df = pd.read_csv("sample_user.csv")

In [14]:
user_df

,Unnamed: 0,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,494127,knX77K5Kof0_j4Bldyqjyw,Beth,3,2016-07-17 01:05:21,0,0,0,NaN,"61cgUglkQH61nj74WWdJyQ, iImaiEExFUEMDRjVJ1mk1w...",...,0,0,0,0,0,0,0,0,0,0
1,265553,MMYlce0L_M65xeh8SwV3WA,Laura,21,2014-09-22 18:46:00,23,10,12,NaN,"tg4TM48hsJ2SclXTdLLvWg, swEagjOeTKwdp9VnZ8PM0Q",...,0,0,0,0,4,1,0,0,0,0
2,195169,fTU0oU6hZR6gPS0MIjFc_w,Steve,193,2015-10-26 01:17:11,37,6,10,NaN,None,...,0,0,0,0,0,0,1,1,1,1
3,98962,IB5f0wf3su09GC9O0754nA,Brianna,16,2014-07-06 03:06:48,39,10,7,NaN,"PXa3G3OEdpn71cztsqHevg, saaI4u6JKWfzUxwFwsn38g...",...,0,0,0,0,0,1,0,0,0,0
4,294551,R2zy34oAFEKthOssSnwY8A,Dana,15,2015-01-11 15:13:30,5,0,0,NaN,"SX8afpQiZ13Ori7FHNhvqw, U3zUbKEgjwyo-j8PTszNnw...",...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,82738,5Uf6gMUssJsaHvLHGH38NQ,Mink,8,2011-10-15 23:46:28,26,7,10,NaN,"fEtWwhNKSoTqIWTSEwGIvg, c8pMZVU9GxiuL-B14vkajg...",...,0,0,0,0,3,1,0,0,0,0
39996,673948,o2NAdWq8oKy9FOMgEcrUnQ,Amy,13,2012-11-28 23:39:16,6,0,1,NaN,"fdQNLtitZiuXBn6F7THwvA, u-LGJKCxVp4SXxL5lRo2dA...",...,0,0,0,0,0,0,0,0,0,0
39997,973699,K15WEci5nx-QvslLM-gH7A,S,2,2019-04-14 16:33:43,1,0,0,NaN,None,...,0,0,0,0,0,0,0,0,0,0
39998,352571,D5QM36MpZKWyTZYWdZ_WGg,Barbara,51,2014-05-18 04:31:52,19,3,4,NaN,hMdZQTD-t9k5JbxmiNt0Uw,...,0,0,0,0,0,1,0,0,0,0


In [15]:
tip_df

,Unnamed: 0,user_id,business_id,text,date,compliment_count
0,729739,KBen7mIEswFkf1WOJUR3Eg,WjCPK_Y67TuiFLLHc_NHoA,"Karen is great! Professional, fast and does a ...",2017-02-16 16:09:45,0
1,307265,xLFJb3n_Hg5iET6799EwuQ,UkzBUVMDXi9iN3l4cGXpWg,Always great food and service but a bit pricey...,2019-01-24 03:28:37,0
2,747815,3LNCN_aMHo97xvEFR4PhDw,qb28j-FNX1_6xm7u372TZA,Try the honeydew daquiri! Avoid the food.,2018-02-13 00:06:51,0
3,331772,X1nvKXUJ5Lp3W9Oe-_JrMQ,cXMdeBlGAbmQRo47rd0s0g,For being one of very very few stations nearby...,2011-07-12 22:48:36,0
4,69814,WtJ_2EyXceJEaVqj14P4TA,SbdL-8NSmTWgSwdGZBa7WQ,Had another great pizza from these guys today....,2017-03-07 01:39:04,0
...,...,...,...,...,...,...
39995,206971,fCvMnJU1Z-XhAjKg99wK3Q,Hr_zdqPZ9VkGjKNv4UIQRg,15% off pickup.,2020-05-12 16:19:07,0
39996,840090,_uY0oABo6aApmuuhR6lkXw,Q8q-1MZTYPL9ZBmAeoMKVg,The owner has two faces over there.,2019-10-26 04:31:34,0
39997,468190,oRFa3bx6JkVxBkvDDe2JeA,2WGnykxiM-Mp-qIm2u7iAw,A little Italian meal before we embark on the ...,2011-07-18 22:19:37,0
39998,274558,7yuBjruAJfk0n0eTFONMSg,LDbmL__o7H6VILMEpxm44A,Everyone keeps mentioning the cyclists as if t...,2014-08-02 16:18:27,0
